In [1]:
import fastgwr
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../Data/2023_edges.csv', header =None)
with open('../Data/2023_edges_header.csv', 'r') as file:
    columns = file.readline().strip().split(',')
df.columns = columns

In [3]:
df_nodes = pd.read_csv('../Data/2023_nodes.csv', header =None)
with open('../Data/2023_nodes_header.csv', 'r') as file:
    columns = file.readline().strip().split(',')
df_nodes.columns = columns
df_nodes

,stationId:ID,name,latitude:float,longitude:float,:LABEL
0,0,Métro Mont-Royal (Utilités publiques / Rivard),45.524236,-73.581552,Station
1,1,Rielle / Wellington,45.460156,-73.567001,Station
2,2,Ste-Catherine / Drummond,45.498588,-73.574278,Station
3,3,Boyer / du Mont-Royal,45.527432,-73.579917,Station
4,4,Clark / Ontario,45.510625,-73.566903,Station
...,...,...,...,...,...
935,2210657,Gilford / Drolet,45.523970,-73.585075,Station
936,4868044,Parc Père-Marquette ( de Bellechasse / de Lana...,45.539354,-73.595374,Station
937,4970813,Saint-Sylvestre / Labonté,45.535426,-73.513659,Station
938,11279502,Jardin Botanique (Sherbrooke / Pie-IX),45.556792,-73.554647,Station


In [4]:
df_edges_extended = pd.merge(df,df_nodes,left_on=':START_ID',right_on='stationId:ID')
df_edges_extended.rename(columns={'latitude:float':'start_latitude', 'longitude:float':'start_longitude'},inplace=True)
df_edges_extended.drop(['name',':LABEL','stationId:ID'],inplace=True,axis=1)
df_edges_extended = pd.merge(df_edges_extended,df_nodes,left_on=':END_ID',right_on='stationId:ID')
df_edges_extended.rename(columns={'latitude:float':'end_latitude', 'longitude:float':'end_longitude'},inplace=True)
df_edges_extended = df_edges_extended[['start_latitude', 'start_longitude','end_latitude','end_longitude', ':START_ID',':END_ID', 'month', 'day', 'hour',
       'temperature', 'wind_speed', 'distance','conditions']]
df_edges_extended

,start_latitude,start_longitude,end_latitude,end_longitude,:START_ID,:END_ID,month,day,hour,temperature,wind_speed,distance,conditions
0,45.524236,-73.581552,45.534134,-73.573524,0,509,10,25,20,18.0,30.0,1.265884,Généralement nuageux
1,45.524236,-73.581552,45.534134,-73.573524,0,509,9,8,1,22.0,15.0,1.265884,Possibilité d'averses. Risque d'orages
2,45.524236,-73.581552,45.534134,-73.573524,0,509,8,28,20,21.0,10.0,1.265884,Partiellement nuageux
3,45.524236,-73.581552,45.534134,-73.573524,0,509,9,23,13,21.0,5.0,1.265884,Généralement ensoleillé
4,45.524236,-73.581552,45.534134,-73.573524,0,509,10,26,0,17.0,15.0,1.265884,Généralement nuageux
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11730057,45.697815,-73.654065,45.702349,-73.639578,55767,12839,11,15,22,8.0,20.0,1.232851,Partiellement nuageux
11730058,45.697815,-73.654065,45.702349,-73.639578,55767,12839,9,21,3,12.0,15.0,1.232851,Dégagé
11730059,45.697815,-73.654065,45.702349,-73.639578,55767,12839,11,15,23,8.0,20.0,1.232851,Partiellement nuageux
11730060,45.617500,-73.606011,45.702349,-73.639578,64993,12839,8,24,0,18.0,10.0,9.788832,Quelques nuages


In [5]:
good_weather = [
    'Quelques nuages', 'Partiellement nuageux', 'Nuageux', 
    'Généralement nuageux. Risque de bruine verglaçante', 'Généralement nuageux', 
    'Généralement ensoleillé', 'Fumée', 'Ensoleillé', 'Dégagé', 
    'Couvert', 'Alternance de soleil et de nuages', 
    'Alternance de soleil et de nuages. Risque d\'orages'
]

# Create a new column to flag good weather
df_edges_extended['good_weather'] = df_edges_extended['conditions'].apply(lambda x: 1 if x in good_weather else 0)

In [6]:
df_edges_extended

,start_latitude,start_longitude,end_latitude,end_longitude,:START_ID,:END_ID,month,day,hour,temperature,wind_speed,distance,conditions,good_weather
0,45.524236,-73.581552,45.534134,-73.573524,0,509,10,25,20,18.0,30.0,1.265884,Généralement nuageux,1
1,45.524236,-73.581552,45.534134,-73.573524,0,509,9,8,1,22.0,15.0,1.265884,Possibilité d'averses. Risque d'orages,0
2,45.524236,-73.581552,45.534134,-73.573524,0,509,8,28,20,21.0,10.0,1.265884,Partiellement nuageux,1
3,45.524236,-73.581552,45.534134,-73.573524,0,509,9,23,13,21.0,5.0,1.265884,Généralement ensoleillé,1
4,45.524236,-73.581552,45.534134,-73.573524,0,509,10,26,0,17.0,15.0,1.265884,Généralement nuageux,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11730057,45.697815,-73.654065,45.702349,-73.639578,55767,12839,11,15,22,8.0,20.0,1.232851,Partiellement nuageux,1
11730058,45.697815,-73.654065,45.702349,-73.639578,55767,12839,9,21,3,12.0,15.0,1.232851,Dégagé,1
11730059,45.697815,-73.654065,45.702349,-73.639578,55767,12839,11,15,23,8.0,20.0,1.232851,Partiellement nuageux,1
11730060,45.617500,-73.606011,45.702349,-73.639578,64993,12839,8,24,0,18.0,10.0,9.788832,Quelques nuages,1


In [7]:
departure_counts = df_edges_extended.groupby([":START_ID","start_longitude","start_latitude","temperature","wind_speed","good_weather","month","day","hour"]).size().reset_index(name='count')
arrival_counts = df_edges_extended.groupby([":END_ID","end_longitude","end_latitude","temperature","wind_speed","good_weather","month","day","hour"]).size().reset_index(name='count')

In [37]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(departure_counts.iloc[:20000],train_size=0.8)
train_departure_variables = train[['temperature','wind_speed','good_weather','month','day','hour']].values
train_departure_locations = train[['start_longitude','start_latitude']].values
train_departure_count = train['count'].values.reshape((-1,1))
test_departure_variables = test[['temperature','wind_speed','good_weather','month','day','hour']].values
test_departure_locations = test[['start_longitude','start_latitude']].values
test_departure_count = test['count'].values.reshape((-1,1))
# arrival_variables = arrival_counts[['temperature','wind_speed','good_weather','month','day','hour']]
# arrival_locations = arrival_counts[['end_longitude','end_latitude']]
# arrival_count = arrival_counts['count']

In [38]:
print(train_departure_locations.shape)
print(train_departure_count.shape)
print(train_departure_variables.shape)
print(test_departure_locations.shape)
print(test_departure_locations.shape)

(16000, 2)
(16000, 1)
(16000, 6)
(4000, 2)
(4000, 2)


In [39]:
#pip install mgwr
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW

In [43]:
model = GWR(train_departure_locations,train_departure_count,train_departure_variables,bw=0.1, kernel='bisquare')
results = model.fit()

In [44]:
results.summary()

Model type                                                         Gaussian
Number of observations:                                               16000
Number of covariates:                                                     7

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                         975030.496
Log-likelihood:                                                  -55582.057
AIC:                                                             111178.113
AICc:                                                            111180.122
BIC:                                                             820212.754
R2:                                                                   0.123
Adj. R2:                                                              0.123

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

In [46]:
prediction = model.predict(test_departure_locations, test_departure_variables)

In [54]:
y_prediction_gwr = prediction.predy
print(y_prediction_gwr)
np.savetxt('C:/Master/Semestre_2/Social_media_analytics/Project/prediction_gwr.csv', y_prediction_gwr, delimiter=',', fmt='%d')


[[ 9.94940092]
 [10.72184701]
 [ 1.53211279]
 ...
 [ 2.37918372]
 [14.49248603]
 [ 1.29318061]]
